In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq


In [2]:
df_sites = pd.read_csv('data/t_sites.csv')
df_metadata = pd.read_excel('data/LEM_Resi_MetaData_Tables.xlsx', sheet_name='Household Questionaire')

In [3]:
df_sites = df_sites[['site','besskwh','besskw','pvkw','majorload']]
df_metadata_ev = df_metadata[['newsite']+list(df_metadata.columns[52:60])].dropna(subset=df_metadata.columns[52:60], how='all')

In [4]:
df_metadata_ev

,newsite,EV Vehicle Make?,What is the main uses for your electric vehicle?,What is the main uses for your electric vehicle?.1,What is the main uses for your electric vehicle?.2,Approx No. of trips per day,Approx No.of trips per week,Avergae Weekly Mileage in Electric Vehicle,Do you have a dedicated charger for your electric vehicle (not a 3-pin plug)?
35,36,Citroen C-zero and Peugeot Ion,NaN,Shopping,Leisure,4.0,28.0,500,Yes
38,39,Renault Zoe Dynamique Intense,NaN,Shopping,NaN,0.6,4.0,150,Yes
42,43,Renault Zoe Dynamique (to be delivered Nov 2017),NaN,Shopping,Leisure,1.0,7.0,100,Yes
54,55,Prius PHEV,Commuting,NaN,NaN,1.0,7.0,Not applicable,No
61,62,Citroen i-on,NaN,Shopping,Leisure,1.0,7.0,200,Yes
63,64,Mobility scooter,NaN,NaN,Leisure,2.0,14.0,25,No
72,73,but we have a charging point for visitors,NaN,NaN,NaN,NaN,NaN,NaN,Yes
83,84,Nissan Leaf,NaN,Shopping,Leisure,1.0,7.0,50,Yes
95,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes


In [42]:
df_sites[(df_sites['site'].isin(df_metadata_ev['newsite']) | df_sites['majorload'].str.contains('EV'))]

,site,besskwh,besskw,pvkw,majorload
34,35,7.5,3.3,3.920,EV
35,36,7.5,3.3,4.000,
38,39,10.0,3.3,2.000,Shower or EV
42,43,10.0,3.3,3.900,EV
54,55,5.0,2.5,2.565,
61,62,5.0,2.5,3.500,Space Heater
63,64,5.0,2.5,3.990,
72,73,5.0,2.5,4.000,
76,77,7.5,3.3,2.500,EV
83,84,7.5,3.3,3.420,


In [56]:
# read file for site=10, year=2019, month = 10

In [66]:
df = pd.read_parquet('data/t_msb1m/site=10/year=2019/month=10')

In [67]:
# groupby dy and hour

df['date_hour'] = df['ts'].dt.floor('h')

df_hourly = df.groupby(['date_hour']).agg({'production_wh':'mean','consumption_wh':'mean'}).reset_index()
df_hourly['production_wh'] = df_hourly['production_wh']*60
df_hourly['consumption_wh'] = df_hourly['consumption_wh']*60

In [68]:
df_hourly

,date_hour,production_wh,consumption_wh
0,2019-10-01 00:00:00,0.0,63.06
1,2019-10-01 01:00:00,0.0,59.53
2,2019-10-01 02:00:00,0.0,53.30
3,2019-10-01 03:00:00,0.0,54.84
4,2019-10-01 04:00:00,0.0,55.80
...,...,...,...
739,2019-10-31 19:00:00,0.0,812.41
740,2019-10-31 20:00:00,0.0,226.54
741,2019-10-31 21:00:00,0.0,222.33
742,2019-10-31 22:00:00,0.0,180.32


In [23]:
date_range = ('2019-10-01', '2020-01-01')
master =[]
for site in df_sites['site']:
    df = pd.read_parquet(f'data/t_msb1m/site={site}/', engine='pyarrow')
    df= df[(df['ts']>=date_range[0]) & (df['ts']<date_range[1])]
    df['site'] = site
    master.append(df)

In [24]:
master_df = pd.concat(master)

In [25]:
master_df

,ts,tz,hour,minute,soc,discharge_wh,charge_wh,production_wh,consumption_wh,gridexport_wh,...,pvexport_wh,griddischarge_wh,gridcharge_wh,gridcons_wh,consdischarge_wh,mismatch_wh,year,month,day,site
9332,2019-10-01 23:59:00,UTC+01:00,23,59,0,0.0,0.13,0.0,2.53,0.0,...,0.0,0.0,0.13,2.53,0.0,0.0,2019,10,1,1
9333,2019-10-01 23:58:00,UTC+01:00,23,58,0,0.0,0.13,0.0,2.53,0.0,...,0.0,0.0,0.13,2.53,0.0,0.0,2019,10,1,1
9334,2019-10-01 23:57:00,UTC+01:00,23,57,0,0.0,0.13,0.0,2.52,0.0,...,0.0,0.0,0.13,2.52,0.0,0.0,2019,10,1,1
9335,2019-10-01 23:56:00,UTC+01:00,23,56,0,0.0,0.13,0.0,2.53,0.0,...,0.0,0.0,0.13,2.53,0.0,0.0,2019,10,1,1
9336,2019-10-01 23:55:00,UTC+01:00,23,55,0,0.0,0.13,0.0,2.53,0.0,...,0.0,0.0,0.13,2.53,0.0,0.0,2019,10,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243956,2019-12-09 00:04:00,UTC+00:00,0,4,3,0.0,0.20,0.0,2.97,0.0,...,0.0,0.0,0.20,2.97,0.0,0.0,2019,12,9,100
243957,2019-12-09 00:03:00,UTC+00:00,0,3,3,0.0,0.20,0.0,3.25,0.0,...,0.0,0.0,0.20,3.25,0.0,0.0,2019,12,9,100
243958,2019-12-09 00:02:00,UTC+00:00,0,2,3,0.0,0.20,0.0,3.62,0.0,...,0.0,0.0,0.20,3.62,0.0,0.0,2019,12,9,100
243959,2019-12-09 00:01:00,UTC+00:00,0,1,3,0.0,0.20,0.0,3.63,0.0,...,0.0,0.0,0.20,3.63,0.0,0.0,2019,12,9,100


In [26]:
count = master_df.groupby('site').agg({'ts':'count'})